In [1]:
import pandas as pd
import numpy as np
import scipy.io
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from datasetResize import *
from sklearn.metrics import classification_report
from keras import layers, Sequential
import matplotlib.pyplot as plt


In [2]:
# lecture du excel
labels_df = pd.read_csv('REFERENCE-V3.csv', header=None)
labels_df.columns = ['filename', 'label']

print(labels_df.head())

  filename label
0   A00001     N
1   A00002     N
2   A00003     N
3   A00004     A
4   A00005     A


In [3]:
# Charger tous les signaux et leurs labels
labels = []
signals = []
signals_padded = []
signals_truncated = []
signals_interpolated = []

# Initialiser avec les paramètres pour le redimensionnement
ecg_resizer_max = ECGResizing(target_length=18286)
ecg_resizer_med = ECGResizing(target_length=9000)

for index, row in labels_df.iterrows():
    
    labels.append(row['label'])
    filename = row['filename']
    # Charger un signal ECG
    signal = ecg_resizer_max.load_ecg(f'training2017/{filename}.mat')
    #signals = np.append(signals,[[signal]])
    signals.append(signal)

    # Appliquer le zero padding
    signal_padded = ecg_resizer_max.resize_signal(signal, method='padding')
    signals_padded.append(signal_padded)

    # Appliquer le tronquage
    signal_truncated = ecg_resizer_med.resize_signal(signal, method='padding')
    signals_truncated.append(signal_truncated)

    # Appliquer l'interpolation
    signal_interpolated = ecg_resizer_med.resize_signal(signal, method='interpolate')
    signals_interpolated.append(signal_interpolated)



In [4]:
print(len(signals[0]))
print(len(signals_padded[0]))
print(len(signals_truncated[0]))
print(len(signals_interpolated[0]))

9000
18286
9000
9000


In [5]:
labels = np.array(labels)
signals_padded = np.array(signals_padded)
signals_truncated = np.array(signals_truncated)
signals_interpolated = np.array(signals_interpolated)


# Encoder les labels
label_mapping = {'N': 0, 'A': 1, 'O': 2, '~': 3}
labels_encoded = np.vectorize(label_mapping.get)(labels)


In [6]:
# Diviser en ensembles d'entraînement et de test
#X_train_signals, X_test_signals, y_train_signals, y_test_signals = train_test_split(signals, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_padded, X_test_padded, y_train_padded, y_test_padded = train_test_split(signals_padded, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_truncated, X_test_truncated, y_train_truncated, y_test_truncated = train_test_split(signals_truncated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)
X_train_interpolated, X_test_interpolated, y_train_interpolated, y_test_interpolated = train_test_split(signals_interpolated, labels_encoded, test_size=0.2, stratify=labels, random_state=42)

# Afficher les formes des ensembles
print(f'Ensemble d\'entraînement : {X_train_padded.shape}, Ensemble de test : {X_test_padded.shape}')

Ensemble d'entraînement : (6822, 18286), Ensemble de test : (1706, 18286)


In [7]:
# Créer le modèle MLP
model_std_18k = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_padded.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_std_18k.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model_std_18k.summary()

c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │     2,340,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,344,996 (8.95 MB)

 Trainable params: 2,344,996 (8.95 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_padded), y=y_train_padded)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history = model_std_18k.fit(X_train_padded, y_train_padded, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2506 - loss: 326.7471 - val_accuracy: 0.2073 - val_loss: 130.1801
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3570 - loss: 52.8116 - val_accuracy: 0.4293 - val_loss: 28.8299
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5825 - loss: 11.3474 - val_accuracy: 0.4806 - val_loss: 17.0228
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5881 - loss: 3.3408 - val_accuracy: 0.4974 - val_loss: 12.1100
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6056 - loss: 2.0985 - val_accuracy: 0.5311 - val_loss: 9.9541
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6427 - loss: 2.0737 - val_accuracy: 0.5267 - val_loss: 9.3018
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6300 - loss: 1.3846 - val_accuracy: 0.5377 - val_loss: 8.2834
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6494 - loss: 1.1498 - val_acc

In [9]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_std_18k.evaluate(X_test_padded, y_test_padded)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_std_18k.predict(X_test_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_padded, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5804 - loss: 11.4943
Précision sur l'ensemble de test : 0.5744
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.95      0.73      1015
           A       0.00      0.00      0.00       152
           O       0.44      0.03      0.06       483
           ~       0.10      0.07      0.08        56

    accuracy                           0.57      1706
   macro avg       0.28      0.26      0.22      1706
weighted avg       0.48      0.57      0.46      1706



In [10]:
# Créer le modèle MLP
model_trnctd_9k = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model_trnctd_9k.summary()

c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │     1,152,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,388 (4.41 MB)

 Trainable params: 1,156,388 (4.41 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k.fit(X_train_truncated, y_train_truncated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2802 - loss: 486.4412 - val_accuracy: 0.2293 - val_loss: 241.8592
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3088 - loss: 144.2455 - val_accuracy: 0.2117 - val_loss: 108.7142
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3277 - loss: 38.8191 - val_accuracy: 0.2264 - val_loss: 68.0321
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3406 - loss: 21.6743 - val_accuracy: 0.2674 - val_loss: 33.7963
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3889 - loss: 4.0727 - val_accuracy: 0.2755 - val_loss: 29.5019
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3798 - loss: 2.7806 - val_accuracy: 0.2689 - val_loss: 24.3305
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3921 - loss: 3.2617 - val_accuracy: 0.2784 - val_loss: 20.9080
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4909 - loss: 1.3271 - v

In [12]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5336 - loss: 15.0817
Précision sur l'ensemble de test : 0.5234
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.61      0.83      0.70      1015
           A       0.16      0.06      0.09       152
           O       0.28      0.07      0.12       483
           ~       0.04      0.11      0.06        56

    accuracy                           0.52      1706
   macro avg       0.27      0.27      0.24      1706
weighted avg       0.46      0.52      0.46      1706



In [13]:
# Créer le modèle MLP
model_intrpld_9k = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_intrpld_9k.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model_intrpld_9k.summary()

c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 128)            │     1,152,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,388 (4.41 MB)

 Trainable params: 1,156,388 (4.41 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_interpolated), y=y_train_interpolated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_interpolated = model_intrpld_9k.fit(X_train_interpolated, y_train_interpolated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2638 - loss: 569.5490 - val_accuracy: 0.2073 - val_loss: 290.4857
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2917 - loss: 141.2966 - val_accuracy: 0.1700 - val_loss: 80.3658
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2533 - loss: 33.1814 - val_accuracy: 0.1370 - val_loss: 40.1557
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4158 - loss: 6.1644 - val_accuracy: 0.4930 - val_loss: 26.8095
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5901 - loss: 3.8078 - val_accuracy: 0.5165 - val_loss: 22.4784
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6032 - loss: 1.9820 - val_accuracy: 0.5223 - val_loss: 19.8843
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6274 - loss: 1.6276 - val_accuracy: 0.5209 - val_loss: 18.5703
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6254 - loss: 1.4602 - val

In [15]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_intrpld_9k.evaluate(X_test_interpolated, y_test_interpolated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_intrpld_9k.predict(X_test_interpolated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_interpolated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5745 - loss: 7.4100
Précision sur l'ensemble de test : 0.5686
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.61      0.92      0.73      1015
           A       0.00      0.00      0.00       152
           O       0.33      0.06      0.10       483
           ~       0.11      0.12      0.12        56

    accuracy                           0.57      1706
   macro avg       0.26      0.28      0.24      1706
weighted avg       0.46      0.57      0.47      1706



# Modification des hyperparamètre sur le truncated 9k

### TEST N°1 truncated: augmentation du nombre d'epoch

In [16]:
# Créer le modèle MLP
model_trnctd_9k_test1 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test1.fit(X_train_truncated, y_train_truncated, epochs=300, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/300


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2383 - loss: 456.1823 - val_accuracy: 0.2630 - val_loss: 241.5354
Epoch 2/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3615 - loss: 143.6525 - val_accuracy: 0.2073 - val_loss: 115.6124
Epoch 3/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3329 - loss: 43.6283 - val_accuracy: 0.1773 - val_loss: 47.5198
Epoch 4/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2899 - loss: 9.1338 - val_accuracy: 0.1788 - val_loss: 32.1455
Epoch 5/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2809 - loss: 8.7735 - val_accuracy: 0.1582 - val_loss: 30.2129
Epoch 6/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2638 - loss: 6.7166 - val_accuracy: 0.1516 - val_loss: 51.3163
Epoch 7/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2401 - loss: 8.4475 - val_accuracy: 0.1370 - val_loss: 21.1251
Epoch 8/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2363 - loss: 2.2078 - val_ac

In [53]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test1.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test1.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0941 - loss: 1.4409
Précision sur l'ensemble de test : 0.0891
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

           N       0.00      0.00      0.00      1015
           A       0.09      0.99      0.16       152
           O       1.00      0.00      0.00       483
           ~       0.00      0.00      0.00        56

    accuracy                           0.09      1706
   macro avg       0.27      0.25      0.04      1706
weighted avg       0.29      0.09      0.02      1706



c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### TEST N°2 truncated: Nombre d'epoch optimum (90)

In [55]:
# Créer le modèle MLP
model_trnctd_9k_test2 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test2.fit(X_train_truncated, y_train_truncated, epochs=50, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/50


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2421 - loss: 465.5719 - val_accuracy: 0.1985 - val_loss: 237.5749
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2553 - loss: 120.9290 - val_accuracy: 0.3011 - val_loss: 90.7833
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4193 - loss: 35.4246 - val_accuracy: 0.3795 - val_loss: 50.4660
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5246 - loss: 15.8973 - val_accuracy: 0.4242 - val_loss: 29.8360
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5521 - loss: 7.5090 - val_accuracy: 0.4769 - val_loss: 23.0603
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6004 - loss: 4.2848 - val_accuracy: 0.4982 - val_loss: 18.8109
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6286 - loss: 1.4980 - val_accuracy: 0.4974 - val_loss: 17.4702
Epoch 8/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6412 - loss: 1.3290 - val_accuracy:

In [56]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test2.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test2.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5913 - loss: 2.9774
Précision sur l'ensemble de test : 0.5897
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.99      0.74      1015
           A       0.25      0.01      0.01       152
           O       0.14      0.00      0.00       483
           ~       0.00      0.00      0.00        56

    accuracy                           0.59      1706
   macro avg       0.25      0.25      0.19      1706
weighted avg       0.42      0.59      0.45      1706



### TEST N°3 truncated: Augmentation du nombre de couches (de une à deux)

In [57]:
# Créer le modèle MLP
model_trnctd_9k_test3 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(64, activation='relu'),  # Couche cachée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Résumé du modèle
model_trnctd_9k_test3.summary()
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test3.fit(X_train_truncated, y_train_truncated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_51 (Dense)                │ (None, 128)            │     1,152,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,162,596 (4.43 MB)

 Trainable params: 1,162,596 (4.43 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.2729 - loss: 201.0033 - val_accuracy: 0.2505 - val_loss: 64.1274
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2510 - loss: 64.1378 - val_accuracy: 0.2088 - val_loss: 26.4301
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2398 - loss: 17.3160 - val_accuracy: 0.2960 - val_loss: 14.6310
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3754 - loss: 6.9054 - val_accuracy: 0.4366 - val_loss: 7.2112
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4624 - loss: 1.7627 - val_accuracy: 0.5648 - val_loss: 4.9453
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4739 - loss: 1.6756 - val_accuracy: 0.4403 - val_loss: 5.5533
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4893 - loss: 1.4484 - val_accuracy: 0.5421 - val_loss: 4.8342
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5140 - loss: 1.3571 - val_accur

In [58]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test3.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test3.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4433 - loss: 4.5311
Précision sur l'ensemble de test : 0.4490
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.67      0.63      1015
           A       0.07      0.14      0.10       152
           O       0.31      0.12      0.18       483
           ~       0.03      0.04      0.03        56

    accuracy                           0.45      1706
   macro avg       0.25      0.24      0.23      1706
weighted avg       0.45      0.45      0.44      1706



In [59]:
# Créer le modèle MLP
model_trnctd_9k_test3_bis = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(64, activation='relu'),  # Couche cachée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test3_bis.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test3_bis.fit(X_train_truncated, y_train_truncated, epochs=50, batch_size=32, validation_split=0.2, class_weight=class_weights)


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.2559 - loss: 239.1834 - val_accuracy: 0.2484 - val_loss: 154.4381
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2675 - loss: 143.1249 - val_accuracy: 0.2410 - val_loss: 108.5305
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3856 - loss: 61.6193 - val_accuracy: 0.2308 - val_loss: 90.9261
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4048 - loss: 46.1062 - val_accuracy: 0.2652 - val_loss: 71.0557
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4315 - loss: 25.5181 - val_accuracy: 0.2088 - val_loss: 59.2021
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4435 - loss: 15.6502 - val_accuracy: 0.2484 - val_loss: 44.5231
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4570 - loss: 13.9171 - val_accuracy: 0.2886 - val_loss: 35.8010
Epoch 8/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4844 - loss: 11.8592

In [60]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test3_bis.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test3_bis.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5824 - loss: 1.9865
Précision sur l'ensemble de test : 0.5809
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.97      0.74      1015
           A       0.06      0.01      0.02       152
           O       0.33      0.00      0.00       483
           ~       0.06      0.02      0.03        56

    accuracy                           0.58      1706
   macro avg       0.26      0.25      0.20      1706
weighted avg       0.46      0.58      0.44      1706



### TEST N°4 truncated: Augmentation du nombre de neurone par couche

In [61]:
# Créer le modèle MLP
model_trnctd_9k_test4 = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Résumé du modèle
model_trnctd_9k_test4.summary()
# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test4.fit(X_train_truncated, y_train_truncated, epochs=20, batch_size=32, validation_split=0.2, class_weight=class_weights)


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_59 (Dense)                │ (None, 256)            │     2,304,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,312,612 (8.82 MB)

 Trainable params: 2,312,612 (8.82 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.2629 - loss: 651.8776 - val_accuracy: 0.3304 - val_loss: 171.5834
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4769 - loss: 64.4707 - val_accuracy: 0.5128 - val_loss: 25.8799
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6008 - loss: 4.1252 - val_accuracy: 0.5516 - val_loss: 15.4352
Epoch 4/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6171 - loss: 2.4273 - val_accuracy: 0.5568 - val_loss: 12.8738
Epoch 5/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6284 - loss: 2.5779 - val_accuracy: 0.5619 - val_loss: 11.5994
Epoch 6/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6365 - loss: 1.3781 - val_accuracy: 0.5619 - val_loss: 11.3584
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6360 - loss: 1.2716 - val_accuracy: 0.5641 - val_loss: 11.0956
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6227 - loss: 1.3445 -

In [62]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test4.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test4.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5403 - loss: 10.0775
Précision sur l'ensemble de test : 0.5457
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.59      0.89      0.71      1015
           A       0.12      0.04      0.06       152
           O       0.23      0.03      0.05       483
           ~       0.07      0.07      0.07        56

    accuracy                           0.55      1706
   macro avg       0.25      0.26      0.22      1706
weighted avg       0.43      0.55      0.45      1706



In [63]:
# Créer le modèle MLP
model_trnctd_9k_test4_bis = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_trnctd_9k_test4_bis.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_truncated), y=y_train_truncated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_truncated = model_trnctd_9k_test4_bis.fit(X_train_truncated, y_train_truncated, epochs=50, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/50


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2638 - loss: 514.3107 - val_accuracy: 0.1919 - val_loss: 154.8622
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2235 - loss: 69.3639 - val_accuracy: 0.4410 - val_loss: 35.4544
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5485 - loss: 10.3234 - val_accuracy: 0.5004 - val_loss: 18.3164
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5856 - loss: 2.2776 - val_accuracy: 0.5201 - val_loss: 15.7895
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6291 - loss: 1.7566 - val_accuracy: 0.5311 - val_loss: 13.6171
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6243 - loss: 1.5295 - val_accuracy: 0.5341 - val_loss: 12.5277
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6353 - loss: 1.1864 - val_accuracy: 0.5377 - val_loss: 11.6272
Epoch 8/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6346 - loss: 1.1642 - val_accuracy:

In [64]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_trnctd_9k_test4_bis.evaluate(X_test_truncated, y_test_truncated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_trnctd_9k_test4_bis.predict(X_test_truncated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_truncated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5902 - loss: 2.9884
Précision sur l'ensemble de test : 0.5897
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.99      0.74      1015
           A       0.00      0.00      0.00       152
           O       0.50      0.00      0.00       483
           ~       0.00      0.00      0.00        56

    accuracy                           0.59      1706
   macro avg       0.27      0.25      0.19      1706
weighted avg       0.50      0.59      0.44      1706



### TEST N°1 interpolated: Augmentation du nombre d'epoch

In [65]:
# Créer le modèle MLP
model_interpltd_9k_test1 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_interpolated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_interpltd_9k_test1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_interpolated), y=y_train_interpolated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_interpolated = model_interpltd_9k_test1.fit(X_train_interpolated, y_train_interpolated, epochs=50, batch_size=32, validation_split=0.2, class_weight=class_weights)


Epoch 1/50


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2465 - loss: 495.0851 - val_accuracy: 0.2125 - val_loss: 278.2266
Epoch 2/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3138 - loss: 147.7290 - val_accuracy: 0.2447 - val_loss: 113.1915
Epoch 3/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3399 - loss: 60.7825 - val_accuracy: 0.2425 - val_loss: 101.9273
Epoch 4/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3738 - loss: 22.6244 - val_accuracy: 0.2916 - val_loss: 47.7620
Epoch 5/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4068 - loss: 6.4646 - val_accuracy: 0.2967 - val_loss: 34.2193
Epoch 6/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4095 - loss: 4.4695 - val_accuracy: 0.4366 - val_loss: 33.0550
Epoch 7/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5658 - loss: 4.0388 - val_accuracy: 0.4725 - val_loss: 27.0284
Epoch 8/50
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5917 - loss: 2.5910 - val_accura

In [66]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_interpltd_9k_test1.evaluate(X_test_interpolated, y_test_interpolated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_interpltd_9k_test1.predict(X_test_interpolated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_interpolated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0893 - loss: 2.1355
Précision sur l'ensemble de test : 0.0879
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

           N       0.00      0.00      0.00      1015
           A       0.09      0.98      0.16       152
           O       0.07      0.00      0.00       483
           ~       0.00      0.00      0.00        56

    accuracy                           0.09      1706
   macro avg       0.04      0.25      0.04      1706
weighted avg       0.03      0.09      0.02      1706



c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### TEST N°2 interpolated: Augmentation du nombre de couches (de une à deux)

In [73]:
# Créer le modèle MLP
model_interpltd_9k_test2 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_interpolated.shape[1],)),  # Couche d'entrée
    layers.Dense(64, activation='relu'),  # Couche cachée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_interpltd_9k_test2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Résumé du modèle
model_interpltd_9k_test2.summary()

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_interpolated), y=y_train_interpolated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_interpolated = model_interpltd_9k_test2.fit(X_train_interpolated, y_train_interpolated, epochs=30, batch_size=32, validation_split=0.2, class_weight=class_weights)


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_80 (Dense)                │ (None, 128)            │     1,152,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,162,596 (4.43 MB)

 Trainable params: 1,162,596 (4.43 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.2769 - loss: 218.2848 - val_accuracy: 0.2278 - val_loss: 118.5630
Epoch 2/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2758 - loss: 90.8396 - val_accuracy: 0.2366 - val_loss: 24.2111
Epoch 3/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2929 - loss: 15.8383 - val_accuracy: 0.2652 - val_loss: 9.8965
Epoch 4/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3172 - loss: 2.4269 - val_accuracy: 0.2630 - val_loss: 12.0606
Epoch 5/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3093 - loss: 2.7246 - val_accuracy: 0.2886 - val_loss: 6.9065
Epoch 6/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4493 - loss: 1.5702 - val_accuracy: 0.5538 - val_loss: 5.2554
Epoch 7/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5585 - loss: 1.2693 - val_accuracy: 0.0945 - val_loss: 5.0875
Epoch 8/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3703 - loss: 1.2110 - val_accu

In [74]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_interpltd_9k_test2.evaluate(X_test_interpolated, y_test_interpolated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_interpltd_9k_test2.predict(X_test_interpolated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_interpolated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5870 - loss: 2.0162
Précision sur l'ensemble de test : 0.5862
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           N       0.60      0.98      0.74      1015
           A       0.00      0.00      0.00       152
           O       0.50      0.01      0.01       483
           ~       0.06      0.04      0.04        56

    accuracy                           0.59      1706
   macro avg       0.29      0.26      0.20      1706
weighted avg       0.50      0.59      0.45      1706



### TEST N°3 interpolated: Augmentation du nombre de neurone par couche

In [75]:
# Créer le modèle MLP
model_interpltd_9k_test3 = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_truncated.shape[1],)),  # Couche d'entrée
    layers.Dense(32, activation='relu'),  # Couche cachée
    layers.Dense(4, activation='softmax')  # Couche de sortie (4 classes)
])


model_interpltd_9k_test3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# Résumé du modèle
model_interpltd_9k_test3.summary()

# Calculer le poids de classe pour gérer le déséquilibre
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_interpolated), y=y_train_interpolated)
class_weights = dict(enumerate(class_weights))

# Entraîner le modèle
history_interpolated = model_interpltd_9k_test3.fit(X_train_interpolated, y_train_interpolated, epochs=30, batch_size=32, validation_split=0.2, class_weight=class_weights)


c:\Users\Albus\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_84 (Dense)                │ (None, 256)            │     2,304,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,312,612 (8.82 MB)

 Trainable params: 2,312,612 (8.82 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2420 - loss: 511.6327 - val_accuracy: 0.1949 - val_loss: 74.9022
Epoch 2/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3864 - loss: 32.6347 - val_accuracy: 0.4681 - val_loss: 20.5146
Epoch 3/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5434 - loss: 4.9647 - val_accuracy: 0.5136 - val_loss: 13.3748
Epoch 4/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5813 - loss: 2.1209 - val_accuracy: 0.5070 - val_loss: 26.5746
Epoch 5/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5251 - loss: 4.2300 - val_accuracy: 0.5480 - val_loss: 11.0929
Epoch 6/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6073 - loss: 1.3597 - val_accuracy: 0.5473 - val_loss: 10.4890
Epoch 7/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6300 - loss: 1.2329 - val_accuracy: 0.5487 - val_loss: 10.2719
Epoch 8/30
171/171 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6365 - loss: 1.1585 - val_ac

In [76]:
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model_interpltd_9k_test3.evaluate(X_test_interpolated, y_test_interpolated)
print(f'Précision sur l\'ensemble de test : {test_accuracy:.4f}')

# Faire des prédictions sur l'ensemble de test
predictions = model_interpltd_9k_test3.predict(X_test_interpolated)
predicted_classes = np.argmax(predictions, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_interpolated, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5609 - loss: 4.1180
Précision sur l'ensemble de test : 0.5645
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
              precision    recall  f1-score   support

           N       0.59      0.94      0.73      1015
           A       0.05      0.01      0.02       152
           O       0.23      0.01      0.01       483
           ~       0.02      0.02      0.02        56

    accuracy                           0.56      1706
   macro avg       0.23      0.25      0.20      1706
weighted avg       0.42      0.56      0.44      1706

